# Imports e inicializações

In [1]:
import pathlib
import tarfile
import pandas as pd
import os
import time
import gc
import pdfplumber

import cld3

import multiprocessing


import numpy as np
import json
import bz2
import argparse

from enum import Enum
from unidecode import unidecode


#from openpyxl import *
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook

import re

##Classe com vários métodos de pre-processamento de texto em português criado pelo grupo F03
import utils.preprocessing_portuguese as preprossPT

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
from nltk.corpus import stopwords
from nltk import bigrams
from sklearn.feature_extraction.text import TfidfVectorizer

from matplotlib import pyplot as plt, rcParams, ticker



from tqdm.notebook import tqdm
#from time import time
import shutil
from shutil import rmtree
from queue import Queue
from zipfile import ZipFile
from pyunpack import Archive
from collections import defaultdict

paths = {
    'mun_doc': pathlib.Path('/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios'),
    'tmp-docs': pathlib.Path('/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp'),
}

root_dir = "/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data"


nome_cidade = ""

In [2]:
# List of files from file: 
# https://docs.google.com/spreadsheets/d/1-YzTYRMJwRM9V6pTPB44CYHkxxsmxyVmnqQPvSmFC6A/edit?usp=sharing
compressed_files = [
    #'288_licitacoes-pirapetinga.tar.xz',
    #'289-licitacoes-coqueiral.zip',
    #'290-licitacoes-cristais.zip',
    #'302-licitacoes-olaria.7z',
    #'304-licitacoes-passa-vinte.tar.gz',
    #'306-licitacoes-arantina.7z',
    #'353-licitacoes-ijaci.7z',
    #'381-licitacoes-sao-bento-abade.zip',
    #'385-ribeirao_vermelho.zip',
    '351-licitacoes-itamarati.zip'
]



# Métodos para caracterização dos arquivos

In [3]:
def word_cloud(txt_page, stop_words, add_path, name, page_number):
    print("Nuvem de palavras")
    
    # Create a WordCloud object
    wordcloud = WordCloud(stopwords=stop_words,
                          background_color="white", 
                          max_words=1000, contour_width=3, 
                          contour_color='steelblue', 
                          width=1920, height=1080)

    # Generate a word cloud
    #print(txt_page)
    wordcloud.generate(txt_page)

    # mostrar a imagem final
    fig, ax = plt.subplots(figsize=(10,6))
    #fig, ax = plt.subplots(figsize=(19.20,10.80))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_axis_off()

    #plt.imshow(wordcloud);
    
    if not os.path.exists(os.path.join(root_dir, "img", add_path)):

        os.makedirs(os.path.join(root_dir, "img",add_path))
    
    #print(root_dir+"/"+"img/"+add_path+"/"+"page"+str(page_number)+"-"+name)
    
    fig.savefig(root_dir+"/"+"img/"+add_path+"/"+"page"+str(page_number)+"-"+name, bbox_inches="tight")
    
    plt.close(fig)

In [4]:
def get_text_percentage(file_name: str) -> float:
    """
    Calculate the percentage of document that is covered by (searchable) text.

    If the returned percentage of text is very low, the document is
    most likely a scanned PDF
    """
    total_page_area = 0.0
    total_text_area = 0.0

    doc = fitz.open(file_name)

    for page_num, page in enumerate(doc):
        total_page_area = total_page_area + abs(page.rect)
        text_area = 0.0
        for b in page.getTextBlocks():
            r = fitz.Rect(b[:4])  # rectangle where block text appears
            text_area = text_area + abs(r)
        total_text_area = total_text_area + text_area
    doc.close()
    return total_text_area / total_page_area

In [5]:
def update_xlsx(filename, sheet_name, page_number_file, metadata, qtd_tables, page1_tamvocab, page1_dist, page2_tamvocab, page2_dist, page3_tamvocab, page3_dist):
    
    # Criar um arquivo em excel se não existir
    # Arquivo para armazenar o resultado da caracterização
    
    print(">> Criar ou abrir arquivo para salvar resultado da caracterização")
    print(os.path.exists(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx'))
          
    if not os.path.exists(os.path.join(root_dir, "caracterizacao")):

        os.makedirs(os.path.join(root_dir, "caracterizacao"))

    if not os.path.exists(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx'):
        
        
        wb = openpyxl.Workbook()
        
        print("Criando um novo workbook...")
        
        ws = wb.active
        
        #std=wb.get_sheet_by_name('Sheet')
        ws = wb["Sheet"]
        #wb.remove_sheet(std)
        #std.remove(std['Sheet'])
        #print("Removendo aba padrão...")
        
        ws.title = 'Geral'
        print("Renomeando aba Geral...")
                
        ws.cell(column=1, row=ws.max_row, value="Cidade")
        ws.cell(column=2, row=ws.max_row, value="Quantidade de Arquivos Corrompidos")
        ws.cell(column=3, row=ws.max_row, value="Quantidade de Arquivos Escaneados")
        
    else:
        #Open an xlsx for reading
        print(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx')
        wb = load_workbook(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx')
        print("Workbook loading...")
    
    
    try:
        #Get the current Active Sheet
        ws = wb[sheet_name]
        
        print("Seleciona a aba: ", sheet_name)
    except KeyError:
        print("The worksheet '{}' does not exist for workbook '{}'. Creating one...".format(sheet_name,
                                                                                        'caracterizacao_docs_'+str(sheet_name)+'.xlsx'))
        ws = wb.create_sheet(sheet_name)
        print("Worksheet '{}' created successfully for workbook '{}'.".format(sheet_name, 'caracterizacao_docs_'+str(sheet_name)+'.xlsx'))
        
        ws.cell(column=1, row=ws.max_row, value="Nome do arquivo")
        ws.cell(column=2, row=ws.max_row, value="Quantidade de páginas")
        ws.cell(column=3, row=ws.max_row, value="Metadados")  
        ws.cell(column=4, row=ws.max_row, value="Quantidade de tabelas")
        ws.cell(column=5, row=ws.max_row, value="Pag. 1 - tamanho do vocab")
        ws.cell(column=6, row=ws.max_row, value="Pag. 1 - distribuição palavras")
        ws.cell(column=7, row=ws.max_row, value="Pag. 2 - tamanho do vocab")
        ws.cell(column=8, row=ws.max_row, value="Pag. 2 - distribuição palavras")
        ws.cell(column=9, row=ws.max_row, value="Pag. 3 - tamanho do vocab")
        ws.cell(column=10, row=ws.max_row, value="Pag. 3 - distribuição palavras")
    
    
    print(ws.max_row)
    
    linha = ws.max_row + 1
    
    ws.cell(column=1, row=linha, value=filename)
    ws.cell(column=2, row=linha, value=page_number_file)
    ws.cell(column=3, row=linha, value=str(metadata))  
    ws.cell(column=4, row=linha, value=qtd_tables)
    ws.cell(column=5, row=linha, value=page1_tamvocab)
    ws.cell(column=6, row=linha, value=str(page1_dist))
    ws.cell(column=7, row=linha, value=page2_tamvocab)
    ws.cell(column=8, row=linha, value=str(page2_dist))
    ws.cell(column=9, row=linha, value=page3_tamvocab)
    ws.cell(column=10, row=linha, value=str(page3_dist))
    
    
    print("Saving file - caracterizacao_docs_", str(sheet_name),".xlsx")
    #save the csb file
    wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(sheet_name)+'.xlsx')

In [6]:
def get_info_pdf(main_path, path, name):
    
    global nome_cidade
    
    page1_tamvocab = 0
    page1_dist = ""
    page2_tamvocab = 0
    page2_dist = ""
    page3_tamvocab = 0
    page3_dist = ""
    
    pdfMetadata = ""
    qtd_pages = 0
    
    flag_escaneado = 0
    
    txt_process = preprossPT.TextPreProcessing()
    
    #Identificar o nome da cidade para criar a pasta para ela
    add_path = main_path.split("/")[-1].split(".")[0]
    nome_cidade = add_path
    

    try: 
        
        
        with pdfplumber.open(path+name) as pdf:

                print("Arquivo: ", name)
                print("Metadados: ", pdf.metadata)
                pdfMetadata = pdf.metadata




                #Count the number os pages
                print("Quantidade de páginas: ", pdf.pages[-1].page_number)
                qtd_pages = pdf.pages[-1].page_number



                for pages in pdf.pages:                                

                    page_text = pages.extract_text()


                    #Página com texto igual a none é um indicativo de página escaneada
                    if page_text is None or cld3.get_language(page_text)[0] != 'pt':
                        print("Arquivo escaneado!!")
                        # Copia os arquivos escaneados para a pasta para verificação
                        if not os.path.exists(os.path.join(root_dir, "escaneados")):

                            os.makedirs(os.path.join(root_dir, "escaneados"))

                        # Copia os arquivos escaneados para a pasta de cada cidade
                        if not os.path.exists(os.path.join(root_dir+"/"+"escaneados", add_path)):

                            os.makedirs(os.path.join(root_dir+"/"+"escaneados", add_path))

                        shutil.copy(path+"/"+name, root_dir+"/"+"escaneados"+"/"+add_path)
                        #count_scanned += 1
                        flag_escaneado = 1
                        break
                    elif pages.page_number <= 3: #Caracterização das 3 primeiras páginas
                        print(">> Page: ", pages.page_number)
                        #print(pages.extract_text())

                        #Pre-processamento do texto

                        #Colocando todo o texto em minúsculo
                        page_text = pages.dedupe_chars().extract_text().lower()


                        page_text = txt_process.remove_pronouns(page_text)

                        page_text = txt_process.remove_reduced_or_contracted_words(page_text)

                        page_text = txt_process.remove_adverbs(page_text)

                        page_text = txt_process.remove_special_characters(page_text)

                        page_text = txt_process.remove_excessive_spaces(page_text)

                        page_text = txt_process.remove_accents(page_text)

                        page_text = txt_process.remove_stopwords(page_text)

                        page_text = txt_process.remove_symbols_from_numbers(page_text)

                        page_text = txt_process.remove_numbers(page_text)

                        page_text = txt_process.remove_urls(page_text)

                        if len(page_text) != 0:

                            #Removendo letras sozinhas no texto
                            page_text = re.sub(r'(?:^| )\w(?:$| )', ' ', page_text).strip()

                            page_text = page_text.replace("_","")

                            #Tamanho do texto com palavras repetidas
                            print("Tamanho do texto em quantidade de caracteres com palavras repetidas: ", len(page_text)) 



                            #Obtendo os itens do vocabulário
                            tokens = nltk.word_tokenize(page_text)

                            #Removendo mais stopwords
                            stop_words = set(stopwords.words('portuguese'))

                            tokens = [w for w in tokens if not w.lower() in stop_words]

                            print(tokens)


                            string_bigrams = bigrams(nltk.Text(tokens))

                            #print(tokens)

                            print("Tamanho do vocabulário: ", len(tokens))

                            #print(list(string_bigrams))

                            fdist1 = nltk.FreqDist(nltk.Text(tokens))

                            #print(fdist1.most_common(50))


                           #print(type(page_text))

                            #word_cloud(page_text, stop_words, add_path, name, pages.page_number)

                            if(pages.page_number == 1):
                                page1_tamvocab = len(tokens)
                                page1_dist = fdist1.most_common(100)
                            elif(pages.page_number == 2):
                                page2_tamvocab = len(tokens)
                                page2_dist = fdist1.most_common(100)
                            elif(pages.page_number == 3):
                                page3_tamvocab = len(tokens)
                                page3_dist = fdist1.most_common(100)


                            #Extrai as tabelas das páginas quando existem    
                            tables = pages.extract_tables()



                if not flag_escaneado:
                    if tables:
                        print("Quantidade de Tabelas: ", len(tables))
                        qtd_tables = len(tables)
                    else:
                        print("Arquivo não possui tabela")
                        qtd_tables = 0


        if not flag_escaneado:
            update_xlsx(name, add_path, qtd_pages, pdfMetadata, qtd_tables, page1_tamvocab, page1_dist, page2_tamvocab, page2_dist, page3_tamvocab, page3_dist)
                



    except: 

        # Copia os arquivos corrompidos para a pasta para verificação
        if not os.path.exists(os.path.join(root_dir, "corrompido")):

            os.makedirs(os.path.join(root_dir, "corrompido"))



        # Copia os arquivos corrompidos para a pasta de cada cidade
        if not os.path.exists(os.path.join(root_dir+"/"+"corrompido", add_path)):

            os.makedirs(os.path.join(root_dir+"/"+"corrompido", add_path))

        shutil.copy(path+"/"+name, root_dir+"/"+"corrompido"+"/"+add_path)

        print(">> Arquivo: ", name)
        print(">> FALHA: Arquivo corrompido!")

        #count_corrompido += 1
        pass
        

In [7]:
def salva_escaneado_corrompido(file, flag):
    #Registrando a quantidade de arquivos corrompidos e escaneados no arquivo
    wb = load_workbook(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(file).split(".")[0]+'.xlsx')
    ws = wb.get_sheet_by_name("Geral")
    ws = wb["Geral"]
    
    if not os.path.exists(root_dir+"/"+"corrompido"+"/"+str(file).split(".")[0]):
        count_corrompido = 0
    else:
        count_corrompido = next(os.walk(root_dir+"/"+"corrompido"+"/"+str(file).split(".")[0]))[2] #dir is your directory path as string
        count_corrompido = len(count_corrompido)
    
    if not os.path.exists(root_dir+"/"+"escaneados"+"/"+str(file).split(".")[0]):
        count_scanned = 0
    else:
        count_scanned = next(os.walk(root_dir+"/"+"escaneados"+"/"+str(file).split(".")[0]))[2] #dir is your directory path as string
        count_scanned = len(count_scanned)
    
    if flag:
        linha = ws.max_row
    else:
        linha = ws.max_row + 1

    ws.cell(column=1, row=linha, value=str(file).split(".")[0])
    ws.cell(column=2, row=linha, value=str(count_corrompido))
    ws.cell(column=3, row=linha, value=str(count_scanned))  

    print("Arquivos escaneados: ", str(count_scanned))
    print("Arquivos corrompidos: ", str(count_corrompido))

    print("Saving file info corrompidos e escaneados - "+'caracterizacao_docs_'+str(file).split(".")[0]+'.xlsx')
    #save the csb file
    wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_'+str(file).split(".")[0]+'.xlsx')
    

# Métodos para acessar os arquivos de cada cidade

In [8]:
def list_by_file_format(path, file_format):
    path = path.rstrip("/") + "/"
    
    path_aux = path+"data/files/"
    try:
        file_descriptor = [json.loads(line) for line in open(path_aux  +"file_description.jsonl").readlines()]
    except: 
        print(os.listdir(path)[0])
        path_aux = path+os.listdir(path)[0]+"/data/files/"
        file_descriptor = [json.loads(line) for line in open(path_aux  +"file_description.jsonl").readlines()]
    
    files = [item["file_name"] for item in file_descriptor if item["type"] in file_format]
    return path_aux, [f"{path}{f}".rstrip(";") for f in files]

def list_pdf(path):
    return list_by_file_format(path, ["application/pdf","pdf", "pdf;"])

In [9]:
def decompress_file(tmp_path, full_filepath, filename, extension):
    '''
    Decompress a file in a specified directory.
    '''
    
    if extension == '.zip':
        file_obj = ZipFile(full_filepath, 'r')
    elif tarfile.is_tarfile(full_filepath):
        file_obj = tarfile.open(full_filepath, 'r')
    elif extension == '.7z':
        file_obj = Archive(full_filepath)
    elif not extension:
        print('Nothing to do, file uncompressed')
        return None
    
    try:
        file_obj.extractall(tmp_path)
    except(FileNotFoundError):
        pass

# Método responsável por chamar as outras funções para caracterizar os arquivos

In [10]:
def get_all_infos_from_docs(file):
    
    
    print('process {} starting doing business'.format(file))

    # Defining a path to the compressed file
    path = os.path.join(paths['mun_doc'], file)
    # Defining a temporary path to extract the file
    #tmp_path = os.path.join(paths['mun_doc'], 'tmp'+str(time.time())[:8])
    tmp_path = os.path.join(paths['mun_doc'], 'tmp-docs')
    #print(tmp_path)
    if not os.path.exists(tmp_path):
        os.mkdir(tmp_path)
    
    tmp_path = os.path.join(tmp_path, str(file).split(".")[0])
    print(tmp_path)
    if not os.path.exists(tmp_path):
        os.mkdir(tmp_path)
    

    # Decompressing
    extension = pathlib.Path(path).suffix
    decompress_file(tmp_path, path, file, extension)

    print("Decompressing...")

    time.sleep(5)

    tmp_path, files = list_pdf(tmp_path)
    
    for name in files:
        print(name.split("/")[-1])
        get_info_pdf(path, tmp_path, name.split("/")[-1])
    

    # Deleting the uncompressed file after we're done
    rmtree(tmp_path)
    gc.collect()

    salva_escaneado_corrompido(file, 0)#O 0 indica que é a primeira escrita no arquivo
    
        
    
        

# Método para recuperar arquivos válidos dos escaneados

In [11]:
def recupera_arquivo_valido(file):
    name_folder = str(file).split(".")[0]
    
    txt_process = preprossPT.TextPreProcessing()
    

    for arq in os.listdir(os.path.join(root_dir+"/"+"escaneados", name_folder)):
        print(root_dir+"/"+"escaneados/"+name_folder+"/"+arq)
        
        count_notPT = 0
        count_none = 0
        
        with pdfplumber.open(root_dir+"/"+"escaneados/"+name_folder+"/"+arq) as pdf:
            qtd_pages = pdf.pages[-1].page_number
            
            print(qtd_pages)

            for pages in pdf.pages:                                
                page_text = pages.extract_text()
                
                #Página com texto igual a none é um indicativo de página escaneada
                if page_text is None:
                    count_none += 1
                elif cld3.get_language(page_text)[0] != 'pt':
                    count_notPT += 1
                    
        if count_none == qtd_pages:
            print("Arquivo: ", arq, "É escaneado! Todas as páginas são none!")
        elif count_notPT == qtd_pages:
            print("Arquivo: ", arq, "É escaneado! Todas as páginas não estão em pt!")
        else:
            print("Arquivo: ", arq, "NÃO É escaneado!")
            
            #try:
            for pages in pdf.pages: 
                page_text = pages.extract_text()
                if pages.page_number <= 3 and page_text is not None:
                    page_text = pages.dedupe_chars().extract_text().lower()


                    page_text = txt_process.remove_pronouns(page_text)

                    page_text = txt_process.remove_reduced_or_contracted_words(page_text)

                    page_text = txt_process.remove_adverbs(page_text)

                    page_text = txt_process.remove_special_characters(page_text)

                    page_text = txt_process.remove_excessive_spaces(page_text)

                    page_text = txt_process.remove_accents(page_text)

                    page_text = txt_process.remove_stopwords(page_text)

                    page_text = txt_process.remove_symbols_from_numbers(page_text)

                    page_text = txt_process.remove_numbers(page_text)

                    page_text = txt_process.remove_urls(page_text)

                    if len(page_text) != 0:

                        #Removendo letras sozinhas no texto
                        page_text = re.sub(r'(?:^| )\w(?:$| )', ' ', page_text).strip()

                        page_text = page_text.replace("_","")

                        #Tamanho do texto com palavras repetidas
                        print("Tamanho do texto em quantidade de caracteres com palavras repetidas: ", len(page_text)) 



                        #Obtendo os itens do vocabulário
                        tokens = nltk.word_tokenize(page_text)

                        #Removendo mais stopwords
                        stop_words = set(stopwords.words('portuguese'))

                        tokens = [w for w in tokens if not w.lower() in stop_words]

                        print(tokens)


                        string_bigrams = bigrams(nltk.Text(tokens))

                        #print(tokens)

                        print("Tamanho do vocabulário: ", len(tokens))

                        #print(list(string_bigrams))

                        fdist1 = nltk.FreqDist(nltk.Text(tokens))


                        if(pages.page_number == 1):
                            page1_tamvocab = len(tokens)
                            page1_dist = fdist1.most_common(100)
                        elif(pages.page_number == 2):
                            page2_tamvocab = len(tokens)
                            page2_dist = fdist1.most_common(100)
                        elif(pages.page_number == 3):
                            page3_tamvocab = len(tokens)
                            page3_dist = fdist1.most_common(100)


                        #Extrai as tabelas das páginas quando existem    
                        tables = pages.extract_tables()


            update_xlsx(arq, name_folder, qtd_pages, pdf.metadata, len(tables), page1_tamvocab, page1_dist, page2_tamvocab, page2_dist, page3_tamvocab, page3_dist)


            os.remove(root_dir+"/"+"escaneados/"+name_folder+"/"+arq)
            #except:
                #print("Erro ao obter informação do arquivo!! Arquivo escaneado!")
                #pass
            
    salva_escaneado_corrompido(file, 1)#O 1 indica que estou corrigindo o os escaneados
            

# Código principal para o multiprocessamento

In [12]:
if __name__ == "__main__":
    
    
    print("Quantidade de cores: ", multiprocessing.cpu_count())
    
    

    #fire off workers
    jobs = []
    
    for file in compressed_files:
        print("Município: ", file)
        #get_all_infos_from_docs(file)
        
        p = multiprocessing.Process(target=get_all_infos_from_docs, args = (file, ))
        jobs.append(p)
        p.start()
        #print("Multiprocessing...")
        
    for proc in jobs:
        proc.join()
        proc.close()
        
        
    for file in compressed_files:
        print("Recuperando escaneados município: ", file)
        
        
        p = multiprocessing.Process(target=recupera_arquivo_valido, args = (file, ))
        jobs.append(p)
        p.start()
        
        
    for proc in jobs:
        proc.join()
        proc.close()

Quantidade de cores:  8
Município:  351-licitacoes-itamarati.zip
process 351-licitacoes-itamarati.zip starting doing business
/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp-docs/351-licitacoes-itamarati
Decompressing...
licitacoes_itamarati_2017
4876eeed6252ed935e1148a2e16e07fa.pdf
Arquivo:  4876eeed6252ed935e1148a2e16e07fa.pdf
Metadados:  {'Title': 'EXTRATO DE HOMOLOGAÇÃO', 'Author': 'Cliente', 'Creator': 'Microsoft® Word 2010', 'CreationDate': "D:20170412140931-03'00'", 'ModDate': "D:20170412140931-03'00'", 'Producer': 'Microsoft® Word 2010'}
Quantidade de páginas:  4
>> Page:  1
Tamanho do texto em quantidade de caracteres com palavras repetidas:  1339
['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'homologacao', 'pregao', 'presencial', 'prc', 'objeto', 'aquisicao', 'confeccao', 'impressos', 'formularios', 'graficos', 'especificados', 'termo', 'referencia', 'anexo', '

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'informacao', 'secretaria', 'fazenda', 'existencia', 'disponibilidade', 'recursos', 'financeiros', 'condicao', 'imposta', 'lei', 'complementar', 'autorizo', 'emissao', 'documento', 'itamarati', 'minas', 'junho', 'hamilton', 'moura', 'filho', 'prefeito', 'municipal', 'parecer', 'setor', 'juridico', 'atendendo', 'determinacoes', 'contidas', 'lei', 'federal', 'decreto', 'municipal', 'lei', 'alteracoes', 'processo', 'licitatorio', 'pregao', 'presencial', 'acordo', 'determinacoes', 'leis', 'apto', 'homologacao', 'tendo', 'vista', 'informacao', 'secretaria', 'fazenda', 'existencia', 'disponibilidade', 'recursos', 'financeiros', 'condicao', 'imposta', 'lei', 'complementar', 'setor', 'juridico', 'data', 'junho', 'homologacao', 'homologo', 'adjudico', 'processo', 'licitatorio', 'autorizando', 'compra', 'produtos', 'emissao', 'documento', 'itamarati', 'minas', 'junho', 'extra

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'extrato', 'homologacao', 'pregao', 'prc', 'objeto', 'aquisicao', 'medicamentos', 'materiais', 'medico', 'hospitalar', 'odontologicos', 'atendimento', 'solicitacao', 'secretaria', 'municipal', 'saude', 'especificacao', 'detalhada', 'constante', 'anexo', 'edital', 'empresas', 'vencedoras', 'med', 'center', 'comercial', 'ltda', 'cnpj', 'valor', 'total', 'quarenta', 'quatro', 'mil', 'oitocentos', 'noventa', 'reais', 'centavo', 'disk', 'med', 'padua', 'distribuidora', 'medicamentos', 'ltda', 'cnpj', 'valor', 'total', 'duzentos', 'quarenta', 'cinco', 'mil', 'oitocentos', 'quarento', 'reais', 'setenta', 'dois', 'centavos', 'plena', 'distribuidora', 'materiais', 'medico', 'hospitalares', 'ltda', 'cnpj', 'valor', 'total', 'quatrocentos', 'trinta', 'mil', 'quinhentos', 'setenta', 'reais', 'sessenta', 'tres', 'centavos', 'condicao', 'pagamento', 'pagamentos', 'serao', 'feitos

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'edital', 'retificado', 'processo', 'licitatorio', 'pregao', 'presencial', 'tipo', 'licitacao', 'menor', 'preco', 'item', 'data', 'realizacao', 'horas', 'local', 'sede', 'prefeitura', 'municipal', 'itamarati', 'minas', 'setor', 'licitacao', 'objeto', 'aquisicao', 'generos', 'alimenticios', 'material', 'limpeza', 'higiene', 'gas', 'cozinha', 'glp', 'preambulo', 'edital', 'exm', 'prefeito', 'municipal', 'municipio', 'itamarati', 'minas', 'mg', 'sr', 'hamilton', 'moura', 'filho', 'pregoeiro', 'municipal', 'haroldo', 'lourenco', 'rocha', 'equipe', 'apoio', 'torna', 'publico', 'acha', 'aberta', 'licitacao', 'modalidade', 'pregao', 'presencial', 'tipo', 'menor', 'preco', 'unitario', 'sera', 'regida', 'lei', 'federal', 'julho', 'decreto', 'municipal', 'aplicando', 'couberem', 'lei', 'complementar', 'disposicoes', 'lei', 'federal', 'junho', 'atualizada', 'normas', 'regulame

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'proposta', 'termos', 'item', 'clausula', 'iv', 'devera', 'ser', 'apresentada', 'datilografada', 'digitada', 'respeitado', 'vernaculo', 'emendas', 'rasuras', 'final', 'ser', 'identificada', 'assinada', 'representante', 'licitante', 'procurador', 'caso', 'juntando', 'procuracao', 'devendo', 'conter', 'item', 'compativel', 'objeto', 'descrito', 'especificacao', 'tecnica', 'completa', 'definida', 'anexo', 'iii', 'devendo', 'indiciar', 'marca', 'produtos', 'ofertados', 'preco', 'total', 'fixo', 'irreajustavel', 'expresso', 'moeda', 'nacional', 'corrente', 'fica', 'contratado', 'obrigado', 'cumprir', 'presente', 'contrato', 'totalidade', 'compromete', 'apresentar', 'data', 'hora', 'marcada', 'equipamentos', 'fazer', 'face', 'fiel', 'cumprimento', 'presente', 'contrato', 'prazo', 'validade', 'proposta', 'podera', 'ser', 'inferior', 'sessenta', 'dias', 'contados', 'data', 

...

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'edital', 'retificado', 'prefeitura', 'municipal', 'itamarati', 'minas', 'pregao', 'presencial', 'prc', 'i', 'l', 'municipio', 'itamarati', 'minas', 'exma', 'prefeita', 'sra', 'tarcilia', 'rodrigues', 'fernandes', 'pregoeiro', 'haroldo', 'lourenco', 'rocha', 'levam', 'conhecimento', 'interessados', 'forma', 'lei', 'federal', 'junho', 'lei', 'federal', 'julho', 'alteracoes', 'posteriores', 'decreto', 'municipal', 'fara', 'realizar', 'licitacao', 'modalidade', 'pregao', 'tipo', 'menor', 'preco', 'item', 'aquisicao', 'medicamentos', 'materiais', 'medico', 'hospitalar', 'odontologicos', 'atendimento', 'solicitacao', 'secretaria', 'municipal', 'saude', 'especificacao', 'detalhada', 'constante', 'anexo', 'edital', 'recebimento', 'envelopes', 'credenciamento', 'participantes', 'local', 'prefeitura', 'itamarati', 'minas', 'av', 'coronel', 'araujo', 'porto', 'centro', 'dia',

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'comprovem', 'possuir', 'documentos', 'habilitacao', 'requeridos', 'capitulo', 'documentacao', 'poderao', 'concorrer', 'pregao', 'consorcios', 'empresas', 'forma', 'constituicao', 'empresas', 'suspensas', 'participar', 'licitacao', 'realizada', 'prefeitura', 'municipal', 'itamarati', 'minas', 'empresas', 'declaradas', 'inidoneas', 'licitar', 'contratar', 'administracao', 'publica', 'enquanto', 'perdurarem', 'motivos', 'punicao', 'credenciamento', 'representante', 'empresa', 'participante', 'devera', 'apresentar', 'pregoeiro', 'efetuar', 'credenciamento', 'participante', 'pregao', 'munido', 'carteira', 'identidade', 'outra', 'equivalente', 'documento', 'poderes', 'manifestar', 'durante', 'reuniao', 'abertura', 'envelopes', 'proposta', 'documentacao', 'relativas', 'pregao', 'credenciamento', 'sera', 'encerramento', 'registro', 'ultimo', 'proponente', 'apresentar', 'en

['prefeitura', 'municipal', 'itamarati', 'minas', 'estado', 'minas', 'gerais', 'cnpj', 'inscricao', 'estadual', 'isento', 'edital', 'estara', 'disponivel', 'retirada', 'setor', 'licitacoes', 'prefeitura', 'municipal', 'itamarati', 'minas', 'localizada', 'av', 'coronel', 'araujo', 'porto', 'centro', 'itamarati', 'minas', 'mg', 'licitacao', 'destina', 'garantir', 'observancia', 'principio', 'constitucional', 'isonomia', 'selecao', 'proposta', 'vantajosa', 'prefeitura', 'municipal', 'itamarati', 'minas', 'legislacao', 'edital', 'presente', 'licitacao', 'publica', 'reger', 'comandos', 'legais', 'seguintes', 'lei', 'federal', 'alteracoes', 'posteriores', 'lei', 'federal', 'alteracoes', 'posteriores', 'lei', 'licitacoes', 'decreto', 'municipal', 'regulamento', 'pregao', 'exigencias', 'edital', 'anexos', 'efeito', 'edital', 'devem', 'ser', 'consideradas', 'definicoes', 'importantes', 'tais', 'pregao', 'modalidade', 'licitacao', 'disputa', 'fornecimento', 'bens', 'servicos', 'comuns', 'feita',

/dados01/workspace/ufmg.f01dcc/py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  after removing the cwd from sys.path.


Arquivos escaneados:  171
Arquivos corrompidos:  0
Saving file info corrompidos e escaneados - caracterizacao_docs_351-licitacoes-itamarati.xlsx


# Código para salvar os dados dos arquivos de cada cidade em um único arquivo

In [14]:
# The below method is used to read data from an active worksheet and store it in memory.
def reader(file, name_sheet):
    #abs_file = os.path.join(path, file)
    wb_cidade = load_workbook(file)
    
    wb_sheet = wb_cidade[name_sheet]
    rows = []
    for row in wb_sheet.iter_rows(min_row=1):
        row_data = []
        for cell in row:
            row_data.append(cell.value)
        # Creating a list of lists, where each list contain a typical row's data
        rows.append(row_data)
    return rows

In [18]:
for file in os.walk(root_dir+"/"+"caracterizacao"):
    files = file[2]
    break ##Para não pegar do cache

    
if not os.path.exists(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx'):
        
    wb = Workbook()

    print("Criando um novo workbook...")

    ws = wb.active

    ws = wb["Sheet"]

    ws.title = 'Geral'
    print("Renomeando aba Geral...")


    ws.cell(column=1, row=ws.max_row, value="Cidade")
    ws.cell(column=2, row=ws.max_row, value="Quantidade de Arquivos Corrompidos")
    ws.cell(column=3, row=ws.max_row, value="Quantidade de Arquivos Escaneados")
    
    wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
       
else:
    #Open an xlsx for reading
    wb = load_workbook(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
    ws = wb["Geral"]

    
#Valores da aba Geral
for f in files:
    
    print(f)
    
    if f != 'caracterizacao_docs_geral.xlsx':
    
        linha = ws.max_row + 1
        
        wb_cidade = load_workbook(root_dir+"/"+"caracterizacao"+"/"+f)

        ws_cidade = wb_cidade["Geral"]
        
        ws = wb["Geral"]

        print(ws_cidade['A2'].value)
        print(ws_cidade['B2'].value)
        print(ws_cidade['C2'].value)



        ws.cell(column=1, row=linha, value=ws_cidade['A2'].value)
        ws.cell(column=2, row=linha, value=ws_cidade['B2'].value)
        ws.cell(column=3, row=linha, value=ws_cidade['C2'].value)
        
        #wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
    
#files = os.listdir(root_dir+"/"+"caracterizacao")
for f in files:
    
    print(f)
    
    if f != 'caracterizacao_docs_geral.xlsx':
        
        wb_cidade = load_workbook(root_dir+"/"+"caracterizacao"+"/"+f)
        
        ws_cidade = wb_cidade["Geral"]
        
        print("Copiando aba files cidade...", ws_cidade['A2'].value)

        rows = reader(root_dir+"/"+"caracterizacao"+"/"+f, ws_cidade['A2'].value)

        ws = wb.create_sheet(ws_cidade['A2'].value)

        for row in rows:
            ws.append(row)


        wb.save(root_dir+"/"+"caracterizacao"+"/"+'caracterizacao_docs_geral.xlsx')
        
    


caracterizacao_docs_290-licitacoes-cristais.xlsx
290-licitacoes-cristais
2
449
caracterizacao_docs_304-licitacoes-passa-vinte.xlsx
304-licitacoes-passa-vinte
0
26
caracterizacao_docs_381-licitacoes-sao-bento-abade.xlsx
381-licitacoes-sao-bento-abade
0
3
caracterizacao_docs_289-licitacoes-coqueiral.xlsx
289-licitacoes-coqueiral
0
41
caracterizacao_docs_353-licitacoes-ijaci.xlsx
353-licitacoes-ijaci
0
53
caracterizacao_docs_288_licitacoes-pirapetinga.xlsx
288_licitacoes-pirapetinga
2
217
caracterizacao_docs_385-ribeirao_vermelho.xlsx
385-ribeirao_vermelho
0
512
caracterizacao_docs_302-licitacoes-olaria.xlsx
302-licitacoes-olaria
1
9
caracterizacao_docs_306-licitacoes-arantina.xlsx
306-licitacoes-arantina
0
609
caracterizacao_docs_geral.xlsx
caracterizacao_docs_351-licitacoes-itamarati.xlsx
351-licitacoes-itamarati
0
171
caracterizacao_docs_290-licitacoes-cristais.xlsx
Copiando aba files cidade... 290-licitacoes-cristais
caracterizacao_docs_304-licitacoes-passa-vinte.xlsx
Copiando aba fil

In [26]:
onlyfiles = next(os.walk(root_dir+"/corrompido/288_licitacoes-pirapetinga"))[2] #dir is your directory path as string
print(onlyfiles)

['cccb7f338e124e74255cb32191db8711.pdf', '52b402cf42556d167ac094f842af5c2a.pdf', 'b531bb4de5fba4bb67662a35f46e8d61.pdf', 'e6e27fa45f34d3a1901da91c3742eab9.pdf', 'c3beb86e1736d594e51296747e95ccd5.pdf', '49dba2eb80578133228dac7eb786dc11.pdf', '98e6ad8ee8d6045db16ad20915ef8a64.pdf', 'dd2473797f37af028f5c103a080fcc96.pdf', 'd2f1b6411e36509db059ee88eada3774.pdf', 'e9c42d7e693db484a73726813a53af08.pdf', '0a40611882a31adccd31d7da601a4eee.pdf', '57b3b270f077dfbe195d8da38c0fc8dd.pdf', '877c19bb3949b73d122e457aea50ec4d.pdf', '2dfbe42456f1bea75cc69709948f0e5d.pdf', '7f489d2dc20904281261673e2eeadc43.pdf', 'd9dbb193917444053c779bad4c5d85c3.pdf', '790a619895ab9db94cb2028fcef072fc.pdf', '3fd32166b1b2a8dac5678e4935adf261.pdf', '0d1fe516c152da0f50ebaed2b457d76e.pdf', '92910c3f482e08c2053179b69c1f5f21.pdf', '558b0c6ef6710f67ca9bd147c65dea2b.pdf', '7eb7811227184dfcd68a0ec8a209c63a.pdf', 'f263d166366df40e42b7d929b078de6b.pdf', '56f3cc089b48dd861f11402884026e23.pdf', '0db26947ee06c7400bab4b25b6900158.pdf',

In [19]:
tmp_path, files = list_pdf('/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp-docs/351-licitacoes-itamarati')

licitacoes_itamarati_2017


FileNotFoundError: [Errno 2] No such file or directory: '/dados01/workspace/ufmg.f01dcc/m03/business_understanding/data/documentos_municipios/tmp-docs/351-licitacoes-itamarati/licitacoes_itamarati_2017/data/files/file_description.jsonl'